# ETL de el dataset de reviews de Google

importamos librerias a utilizar

In [36]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

### Comenzamos con el dataset de reviews de Google

Extraemos los datos

In [37]:
# Especifica la ruta de la carpeta donde están los archivos JSON
folder_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\Proyecto Final\datasets\Google Maps\review-estados\review-California'

# Obtiene los todos los archivos de la carpeta
files = os.listdir(folder_path)
dataframes = []

# Leer cada archivo JSON línea por línea
for file_name in files:
    if file_name.endswith('.json'):  # Asegurarse de que solo se lean los archivos JSON
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON línea por línea
            dataframes.append(df)
        except ValueError as e:
            print(f"Error leyendo {file_name}: {e}")

# Concatenar todos los DataFrames en uno solo
df_reviews_g_base = pd.concat(dataframes, ignore_index=True)
df_reviews_g_base

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
...,...,...,...,...,...,...,...,...
2699995,1.113423e+20,Byunguk Kim,1573362674409,5,None,None,None,0x80c2bea30829f279:0x39aa953ee93734ed
2699996,1.080818e+20,Ceveda Craytonhooks,1580608853411,5,Beautiful salon. The staff and atmosphere were...,None,None,0x8094675073616747:0x9f935a9b9046a9ba
2699997,1.175914e+20,Natacha Thompson,1528597806165,5,Flo is a great beautician. She is very patie...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1528598932831, 'text': 'Thank you! I...",0x8094675073616747:0x9f935a9b9046a9ba
2699998,1.124266e+20,Eleanor Aikins,1551203379295,5,I Been going to have a variety of hair style a...,None,None,0x8094675073616747:0x9f935a9b9046a9ba


In [38]:
df_reviews_g = df_reviews_g_base 

In [39]:
len(df_reviews_g)

2700000

ETL Basico tambien realizado al comienzo del EDA sobre este mismo dataset

In [40]:
# Eliminar la columna 'pics' ya que no la necesitamos para este analisis
df_reviews_g.drop(columns=['pics'], inplace=True)

# Convertir los diccionarios en la columna 'resp' a cadenas de texto
df_reviews_g['resp'] = df_reviews_g['resp'].apply(lambda x: str(x) if isinstance(x, dict) else x)

# Convertir la columna 'time' a formato de fecha
df_reviews_g['time'] = pd.to_datetime(df_reviews_g['time'], unit='ms')

# Eliminar duplicados
df_reviews_g.drop_duplicates(inplace=True)

Transformamos time y eliminamos fechas previas a 2007 

In [41]:
# Asegurarse de que la columna 'time' esté en formato datetime
df_reviews_g['time'] = pd.to_datetime(df_reviews_g['time'], errors='coerce')

# Detectar outliers en 'time' (fechas fuera del rango)
min_date = pd.to_datetime('2007-06-01')
max_date = pd.to_datetime('today')

# Filtrar las filas que tienen fechas fuera de este rango
outliers_time = df_reviews_g[(df_reviews_g['time'] < min_date) | (df_reviews_g['time'] > max_date)]

# Eliminamos estos outliers
df_reviews_g = df_reviews_g[(df_reviews_g['time'] >= min_date) & (df_reviews_g['time'] <= max_date)]

In [42]:
len(df_reviews_g)

2624574

### Guardamos el dataset en parquet



In [43]:
df_reviews_g.to_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\reviews_google.parquet')

### Filtramos los registros relacionados a locales gastronomicos

A partir del dataframe df_reviews_g sacamos una lista de la columna gmap_id con valores únicos. De esa forma, esa lista permitio filtrar los registros pertenecientes al estado de California en el dataset metadatos de google.
Una vez que tenemos aquellos registros definidos, en los que se ha delimitado el dataset al rubro gastronomico y afines, corresponde ahora volver a filtrar este dataset para que solo queden a disposición los registros del estado de california vinculados al sector gastronomico, lo que haremos a continuación.

In [69]:
import pandas as pd
df_reviews_g = pd.read_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\reviews_google.parquet')

In [60]:
len(df_reviews_g)

2624574

In [70]:
# Cargar la columna 'gmap_id' desde el archivo Parquet
parquet_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\locales_gastronomicos_google.parquet'
gmap_id_california_gastronomico = pd.read_parquet(parquet_path, columns=['gmap_id'])

# Filtrar los registros en df_reviews_g que coincidan en la columna 'gmap_id'
df_reviews_g_filtered = df_reviews_g[df_reviews_g['gmap_id'].isin(gmap_id_california_gastronomico['gmap_id'])]


In [62]:
len(df_reviews_g_filtered)

736570

In [71]:
df_reviews_g_filtered.head()

,user_id,name,time,rating,text,resp,gmap_id
0,1.089912e+20,Song Ro,2021-01-06 05:12:07.056,5,Love there korean rice cake.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-02-09 05:47:28.663,5,Good very good,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-03-08 05:04:42.296,4,They make Korean traditional food very properly.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-03-07 05:56:56.355,5,Short ribs are very delicious.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,2017-05-16 05:01:41.933,5,Great food and prices the portions are large,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


### Eliminamos la columna "resp" ya que determinamos que no aporta valor

In [72]:
# Eliminar la columna 'resp' ya que no la necesitamos para este analisis
df_reviews_g_filtered = df_reviews_g_filtered.drop(columns=['resp'])

In [ ]:
# Reemplazar valores nulos en la columna 'review_text' con 'No review'
df_reviews_g_filtered['text'].fillna('Sin Reseñas', inplace=True)

In [74]:
df_reviews_g_filtered.head()

,user_id,name,time,rating,text,gmap_id
0,1.089912e+20,Song Ro,2021-01-06 05:12:07.056,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-02-09 05:47:28.663,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-03-08 05:04:42.296,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-03-07 05:56:56.355,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,2017-05-16 05:01:41.933,5,Great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49


### Guardamos el Dataframe final en parquet

In [75]:
df_reviews_g_filtered.to_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\reviews_gastronomicos_google.parquet')